# Pengambilan keputusan dengan pemrograman tradisional

In [ ]:
# Daftar kata positif dan negatif
positive_words = ["baik", "bagus", "luar biasa", "hebat", "menyenangkan", "suka"]
negative_words = ["buruk", "jelek", "menyedihkan", "benci", "kecewa", "payah"]

def analyze_sentiment(text):
    text = text.lower()
    pos_count = sum(1 for word in positive_words if word in text)
    neg_count = sum(1 for word in negative_words if word in text)

    if pos_count > neg_count:
        return "Positif"
    elif neg_count > pos_count:
        return "Negatif"
    else:
        return "Netral"

# Contoh penggunaan
text = "Saya sangat suka film ini, luar biasa!"
print(analyze_sentiment(text))  # Output: Positif

# Pengambilan keputusan dengan machine learning

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

# Membaca data dari file CSV
file_path = "data_sentimen.csv"  # Ganti dengan lokasi file Anda
df = pd.read_csv(file_path)

# Memisahkan teks dan label
texts = df["teks"].values
labels = df["label"].values

# Konversi teks menjadi fitur numerik
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(texts)

# Training model Naive Bayes
model = MultinomialNB()
model.fit(X, labels)

# Contoh prediksi dari input baru
test_text = ["film ini sangat tidak bagus"]
test_X = vectorizer.transform(test_text)
print("Hasil Analisis Machine Learning:", model.predict(test_X)[0])

# Model machine learning multilingual sentiment

In [18]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

model_name = "tabularisai/multilingual-sentiment-analysis"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

def predict_sentiment(texts):
    inputs = tokenizer(texts, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
    sentiment_map = {0: "Very Negative", 1: "Negative", 2: "Neutral", 3: "Positive", 4: "Very Positive"}
    return [sentiment_map[p] for p in torch.argmax(probabilities, dim=-1).tolist()]

texts = [
    # English
    "I absolutely love the new design of this app!",
    "The customer service was disappointing.",
    "The weather is fine, nothing special.",
    # Malay
    "Saya suka makanan di sini, sangat sedap!",
    "Pengalaman ini sangat mengecewakan.",
    "Hari ini cuacanya biasa sahaja.",
    # Indonesian
    "Saya tidak suka film ini"
]

for text, sentiment in zip(texts, predict_sentiment(texts)):
    print(f"Text: {text}\nSentiment: {sentiment}\n")


Text: I absolutely love the new design of this app!
Sentiment: Very Positive

Text: The customer service was disappointing.
Sentiment: Negative

Text: The weather is fine, nothing special.
Sentiment: Neutral

Text: Saya suka makanan di sini, sangat sedap!
Sentiment: Very Positive

Text: Pengalaman ini sangat mengecewakan.
Sentiment: Very Negative

Text: Hari ini cuacanya biasa sahaja.
Sentiment: Neutral

Text: Saya tidak suka film ini
Sentiment: Negative

